In [45]:
from sklearn.preprocessing import MultiLabelBinarizer
import json

# 获取数据中所有标签
def get_train_label(path, dataset):
    train = json.load(open(path))
    train_labels = train['labels']
    mlb_label = MultiLabelBinarizer()
    train_labels = mlb_label.fit_transform(train_labels)
    labels = list(mlb_label.classes_)

    with open('/home/oem/projects/lishuqin/coding/data/{}/{}/labels.json'.format(dataset, dataset), 'w') as f:
        json.dump(labels, f, indent=2)

# dataset = 'Eurlex'
# path = '/home/oem/projects/lishuqin/coding/data/Eurlex/Eurlex/train_new.json'

dataset = 'RCV1-V2'
path = '/home/oem/projects/lishuqin/coding/data/RCV1-V2/RCV1-V2/train_new.json'
get_train_label(path, dataset)

In [1]:
# 清理 Eurlex 数据集
# 去除标签为空的数据内容
import json


def removeNone(dataset, train_path, val_path=None, test_path=None):
    train = json.load(open(train_path))
    train_content = train['content']
    train_labels = train['labels']
    print(len(train_labels))           # 15450
    assert len(train_labels) == len(train_content)

    if val_path:
        val = json.load(open(val_path))
        val_content = val['content']
        val_labels = val['labels']
        print(len(val_labels))
        assert len(val_labels) == len(val_content)

    test = json.load(open(test_path))
    test_content = test['content']
    test_labels = test['labels']
    print(len(test_labels))           # 3866
    assert len(test_labels) == len(test_content)

    # 对于Eurlex, 将其中 2164 / 2164.0 处理为一致
    if dataset == 'Eurlex':
        for i in train_labels:
            if '2164' in i:
                i.remove('2164')
                i.append('2164.0')
            if '5641' in i:
                i.remove('5641')
                i.append('5641.0')

    for i in train_content:
        # 内容为空
        if '' == i:
            train_labels.pop(train_content.index(i))
            train_content.remove(i)
    print(len(train_content))
    assert len(train_labels) == len(train_content)

    for i in train_labels:
        # 标签为空，且仅有其一个标签
        if '' in i and len(i) == 1:
            train_content.pop(train_labels.index(i))
            train_labels.remove(i)
        elif '' in i:
            i.remove('')
    print(len(train_content))
    assert len(train_labels) == len(train_content)


    for i in test_content:
        # 内容为空
        if '' == i:
            test_labels.pop(test_content.index(i))
            test_content.remove(i)
    print(len(test_content))
    assert len(test_labels) == len(test_content)

    for i in test_labels:
        # 标签为空，且仅有其一个标签
        if '' in i and len(i) == 1:
            test_content.pop(test_labels.index(i))
            test_labels.remove(i)
        elif '' in i:
            i.remove('')
    print(len(test_content))
    assert len(test_labels) == len(test_content)

    if val_path:
        for i in val_content:
            # 内容为空
            if '' == i:
                val_labels.pop(val_content.index(i))
                val_content.remove(i)
        print(len(val_content))
        assert len(val_labels) == len(val_content)

        for i in val_labels:
            # 标签为空，且仅有其一个标签
            if '' in i and len(i) == 1:
                val_content.pop(val_labels.index(i))
                val_labels.remove(i)
            elif '' in i:
                i.remove('')
        print(len(val_content))
        assert len(val_labels) == len(val_content)


    save_data(train_content, train_labels, 'train', dataset)
    save_data(test_content, test_labels, 'test', dataset)
    if val_path:
        save_data(val_content, val_labels, 'val', dataset)


def save_data(train_content, train_labels, mode, dataset):
    # 保存回去
    train_data = {'content':[], 'labels':[]}
    train_data['content'] = train_content
    train_data['labels'] = train_labels
    with open('/data/lishuqin/data/{}/{}_new.json'.format(dataset, mode), 'w') as f:
        json.dump(train_data, f, indent=2)

    
# dataset = 'Eurlex'
# train = '/data/lishuqin/data/Eurlex/train.json'
# test = '/data/lishuqin/data/Eurlex/test.json'

dataset = 'RCV1-V2'
train = '/data/lishuqin/data/RCV1-V2/train.json'
val = '/data/lishuqin/data/RCV1-V2/val.json'
test = '/data/lishuqin/data/RCV1-V2/test.json'

removeNone(dataset, train, val, test)

775220
21510
1191
775220
775220
1191
1191
21510
21510


In [31]:
# 对标签信息进行扩充，获取更丰富的标签知识
from lm_api import lmp

def rich_label(labels_path, dataset, label_detail_path):
    # label = '7986.0'
    # prompt = 'The following is a tag from the Eurlex dataset. Please explain the meaning of this tag: {}. Requirements: 1. Only answer the meaning. 2. If the tag is composed of an ID, please indicate its corresponding specific legal topic and category.'.format(label)
    # print(prompt)

    with open(labels_path, 'r') as f:
        labels = json.loads(f.read())

    with open(label_detail_path, 'r') as f:
        con = json.loads(f.read())
    for label in labels:
        prompt = 'The following is a tag from the {} dataset. Please explain the meaning of this tag: {}. Requirements: 1. Only answer the meaning. 2. If the tag is composed of an ID, please indicate its corresponding specific legal topic and category. 3. Less than 30 tokens.'.format(dataset, label)
        label_info = lmp(prompt)
        print(label_info)
        with open(label_detail_path, 'w') as f:
            con[label] = label_info
            json.dump(con, f, indent=2)

# dataset = 'Eurlex'
# labels_path = '/home/oem/projects/lishuqin/coding/data/Eurlex/Eurlex/labels.json'
# label_detail_path = '/home/oem/projects/lishuqin/coding/data/Eurlex/Eurlex/label_detail.json'

dataset = 'RCV1-V2'
labels_path = '/home/oem/projects/lishuqin/coding/data/RCV1-V2/RCV1-V2/labels.json'
label_detail_path = '/home/oem/projects/lishuqin/coding/data/RCV1-V2/RCV1-V2/label_detail.json'

rich_label(labels_path, dataset, label_detail_path)



3799


In [59]:
# 删除生成数据的前部分内容
with open('/home/oem/projects/lishuqin/coding/data/Eurlex/Eurlex/label_detail.json', 'r') as f:
    data = json.loads(f.read())

# data = {
#     'dog' : 'The tag refers to abjsahndioshdi',
#     'cat' : 'The big tag refers to abjsadedadsahndioshdi dadawdwq',
#     'rabbit' : 'The tag indicate that bdaih dhnwkadki bja',
#     'monkey' : 'ddgh djia dwidjio'
# }

prefix = ['refers to ', 'indicate that ', 'indicating ']

new = {}
for k, v in data.items():
    for i in prefix:
        if v.find(i) != -1:       # 找到了
            f = v.index(i)
            new[k] = v[f+len(i):]
            break
        else:
            new[k] = v
    
# print(new)

with open('/home/oem/projects/lishuqin/coding/data/Eurlex/Eurlex/label_detail_1.json', 'w') as f:
    json.dump(new, f, indent=2)

In [63]:
data = {
    'dog' : 'The tag refers to abjsahndioshdi',
    'cat' : 'The big tag refers to abjsadedadsahndioshdi dadawdwq',
    'rabbit' : 'The tag indicate that bdaih dhnwkadki bja',
    'monkey' : 'ddgh djia dwidjio'
}

print(list(k + ', ' + v for k, v in data.items()))

['dog The tag refers to abjsahndioshdi', 'cat The big tag refers to abjsadedadsahndioshdi dadawdwq', 'rabbit The tag indicate that bdaih dhnwkadki bja', 'monkey ddgh djia dwidjio']
